In [29]:
import cv2
from yolo_segmentation import YOLOSegmentation
import numpy as np
import os

In [30]:
def rszImg(image, rate):
    h, w, _ = image.shape
    new_h, new_w = (int(h*rate), int(w*rate))
    image = cv2.resize(image, (new_w, new_h))
    return image

def cropPoly(img, bbox, seg):
    (x, y, x2, y2) = bbox
    # Create a mask of the same size as the image
    mask = np.zeros_like(img)

    # Convert the seg (polygon) into a NumPy array of type int32
    points = np.array(seg, dtype=np.int32)

    # Fill the mask with a white polygon based on the points
    cv2.fillPoly(mask, [points], (255, 255, 255))

    # Bitwise AND the mask with the original image to get the cropped region
    cropped_motorcycle = cv2.bitwise_and(img, mask)

    # Crop the region inside the bounding box
    cropped_motorcycle = cropped_motorcycle[y:y2, x:x2]
    
    return cropped_motorcycle

In [31]:
dir_on = {
    'beat':'../dataset-baru/beat-nyala', 'bull':'../dataset-baru/bull-nyala', 'cbr':'../dataset-baru/cbr-nyala', 
    'mio':'../dataset-baru/mio-nyala', 'mio2':'../dataset-baru/mio2-nyala', 'scoopy':'../dataset-baru/scoopy-nyala', 
    'supra':'../dataset-baru/supra-nyala', 'beat_gelap':'../dataset-baru/beat-gelap-nyala/', 'cbr_gelap':'../dataset-baru/cbr-gelap-nyala/'
}

dir_off = {
    'beat':'../dataset-baru/beat-mati', 'bull':'../dataset-baru/bull-mati', 'cbr':'../dataset-baru/cbr-mati', 
    'mio':'../dataset-baru/mio-mati', 'mio2':'../dataset-baru/mio2-mati', 'scoopy':'../dataset-baru/scoopy-mati', 
    'supra':'../dataset-baru/supra-mati', 'beat_gelap':'../dataset-baru/beat-gelap-mati/', 'cbr_gelap':'../dataset-baru/cbr-gelap-mati/'
}

In [32]:
def rszImg(image, rate):
    h, w, _ = image.shape
    new_h, new_w = (int(h*rate), int(w*rate))
    image = cv2.resize(image, (new_w, new_h))
    return image

def cropPoly(img, bbox, seg):
    (x, y, x2, y2) = bbox
    # Create a mask of the same size as the image
    mask = np.zeros_like(img)

    # Convert the seg (polygon) into a NumPy array of type int32
    points = np.array(seg, dtype=np.int32)

    # Fill the mask with a white polygon based on the points
    cv2.fillPoly(mask, [points], (255, 255, 255))

    # Bitwise AND the mask with the original image to get the cropped region
    cropped_motorcycle = cv2.bitwise_and(img, mask)

    # Crop the region inside the bounding box
    cropped_motorcycle = cropped_motorcycle[y:y2, x:x2]
    
    return cropped_motorcycle

def segmentMotorcycle(path):
    global ys
    
    img = cv2.imread(path)
    img = rszImg(img, 0.25)
    bboxes, classes, class_names, segmentation, scores = ys.detect(img)

    cropped_motorcycle_regions = []
    n = 0

    for bbox, class_id, seg, score in zip(bboxes, classes, segmentation, scores):
        # print("bbox:", bbox, " class_id:", class_id, " seg:", seg, " score:", score)
        if class_names[class_id] == 'motorcycle':
            (x, y, x2, y2) = bbox

            cropped_motorcycle = cropPoly(img, bbox, seg)
            cropped_motorcycle_regions.append(cropped_motorcycle)
    return cropped_motorcycle_regions

def saveCrop(filename, cropped_motorcycle_regions):
    if 'super' in filename:
        pass
    else:
        for i, cropped_motorcycle in enumerate(cropped_motorcycle_regions):
            if 'gelap' in filename:
                if 'nyala' in filename:
                    path = 'res/gelap/nyala'
                else:
                    path = 'res/gelap/mati'
            else:
                if 'nyala' in filename:
                    path = 'res/terang/nyala'
                else:
                    path = 'res/terang/mati'
                    
            cv2.imwrite(f'{path}/{filename}', cropped_motorcycle)
    
        
            
def generateDatasetsPoly(directory_path):
    # Iterate through all files and subdirectories in the given directory
    global ys
    for root, _, files in os.walk(directory_path):
        # n=0
        print(directory_path)
        for filename in files:
            # print(filename)
            img = f'{directory_path}/{filename}'
            try:
                cropped_motorcycle_list = segmentMotorcycle(img)
                # print(cropped_motorcycle_list)
                saveCrop(filename, cropped_motorcycle_list)
            except:
                pass

In [33]:
# Segmentation
global ys 
ys = YOLOSegmentation('../../yolov8n-seg.pt')


In [34]:
on_keys = list(dir_on.keys())
off_keys = list(dir_off.keys())

print(on_keys)

['beat', 'bull', 'cbr', 'mio', 'mio2', 'scoopy', 'supra', 'beat_gelap', 'cbr_gelap']


In [35]:
dir_on['beat']

'../dataset-baru/beat-nyala'

In [36]:
for i in on_keys:
    generateDatasetsPoly(dir_on[i])

../dataset-baru/beat-nyala



0: 640x480 1 person, 1 car, 1 motorcycle, 282.5ms
Speed: 5.4ms preprocess, 282.5ms inference, 9.5ms postprocess per image at shape (1, 3, 640, 480)
WARNING  'Masks.segments' is deprecated. Use 'Masks.xyn' for segments (normalized) and 'Masks.xy' for segments (pixels) instead.

0: 640x480 1 person, 1 motorcycle, 288.1ms
Speed: 4.6ms preprocess, 288.1ms inference, 8.0ms postprocess per image at shape (1, 3, 640, 480)
WARNING  'Masks.segments' is deprecated. Use 'Masks.xyn' for segments (normalized) and 'Masks.xy' for segments (pixels) instead.

0: 640x480 1 motorcycle, 333.9ms
Speed: 4.5ms preprocess, 333.9ms inference, 4.6ms postprocess per image at shape (1, 3, 640, 480)
WARNING  'Masks.segments' is deprecated. Use 'Masks.xyn' for segments (normalized) and 'Masks.xy' for segments (pixels) instead.

0: 640x480 1 motorcycle, 401.8ms
Speed: 9.1ms preprocess, 401.8ms inference, 7.0ms postprocess per image at shape (1, 3, 640, 480)
WARNING  'Masks.segments' is deprecated. Use 'Masks.xyn' f

../dataset-baru/bull-nyala



0: 640x480 2 persons, 2 motorcycles, 273.5ms
Speed: 5.6ms preprocess, 273.5ms inference, 10.6ms postprocess per image at shape (1, 3, 640, 480)
WARNING  'Masks.segments' is deprecated. Use 'Masks.xyn' for segments (normalized) and 'Masks.xy' for segments (pixels) instead.

0: 640x480 2 persons, 1 car, 2 motorcycles, 374.6ms
Speed: 7.2ms preprocess, 374.6ms inference, 13.7ms postprocess per image at shape (1, 3, 640, 480)
WARNING  'Masks.segments' is deprecated. Use 'Masks.xyn' for segments (normalized) and 'Masks.xy' for segments (pixels) instead.

0: 640x480 2 persons, 1 motorcycle, 272.0ms
Speed: 3.0ms preprocess, 272.0ms inference, 8.0ms postprocess per image at shape (1, 3, 640, 480)
WARNING  'Masks.segments' is deprecated. Use 'Masks.xyn' for segments (normalized) and 'Masks.xy' for segments (pixels) instead.

0: 640x480 2 persons, 1 motorcycle, 250.0ms
Speed: 4.0ms preprocess, 250.0ms inference, 8.4ms postprocess per image at shape (1, 3, 640, 480)
WARNING  'Masks.segments' is d

../dataset-baru/cbr-nyala



0: 640x480 1 person, 1 motorcycle, 248.7ms
Speed: 4.2ms preprocess, 248.7ms inference, 7.5ms postprocess per image at shape (1, 3, 640, 480)
WARNING  'Masks.segments' is deprecated. Use 'Masks.xyn' for segments (normalized) and 'Masks.xy' for segments (pixels) instead.

0: 640x480 2 persons, 1 motorcycle, 273.7ms
Speed: 4.0ms preprocess, 273.7ms inference, 8.0ms postprocess per image at shape (1, 3, 640, 480)
WARNING  'Masks.segments' is deprecated. Use 'Masks.xyn' for segments (normalized) and 'Masks.xy' for segments (pixels) instead.

0: 640x480 1 person, 2 cars, 1 motorcycle, 254.0ms
Speed: 4.6ms preprocess, 254.0ms inference, 9.1ms postprocess per image at shape (1, 3, 640, 480)
WARNING  'Masks.segments' is deprecated. Use 'Masks.xyn' for segments (normalized) and 'Masks.xy' for segments (pixels) instead.

0: 640x480 1 person, 1 motorcycle, 256.0ms
Speed: 4.1ms preprocess, 256.0ms inference, 6.2ms postprocess per image at shape (1, 3, 640, 480)
WARNING  'Masks.segments' is depreca

AttributeError: 'NoneType' object has no attribute 'segments'

In [ ]:
for i in on_keys:
    generateDatasetsPoly(dir_off[i])